In [61]:
!pip install --upgrade pythainlp
!pip install --upgrade pyLDAvis

In [62]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [63]:
df = pd.read_csv('https://raw.githubusercontent.com/JumpNatchanon/MADT8101-Customer-Analytics/main/05%20Voice%20of%20Customer/Dataset/Dataset_JayFai.csv?token=GHSAT0AAAAAACGRXKS34GQUVXB7KXUI2ADYZHEYVAA')


In [64]:
df.tail()

,Review_ID,Review
15,16,ร้านเจ๊ไฝเป็นร้านที่เราอยากมาลองตั้งแต่ก่อนที่...
16,17,ไข่เจียวปูเจ๊ไฝ นี่มันปูห่อไข่ชัดๆๆ ข้างในคือม...
17,18,คนที่บอกว่าเป็นร้านมิชลินที่ดังที่สุดในโลกไม่ใ...
18,19,เป็นอีก 1 ร้าน ที่กระแสแรงมากๆในปีที่ผ่านมา ขอ...
19,20,กินรอบสองก็ยังอร่อยเหมือนเดิม อร่อยไม่ไหวแล้ว ...


In [65]:
stopwords=list(pythainlp.corpus.thai_stopwords())
removed_words=[' ', ' ', '\n', 'ไป', 'ที่', 'ครับ', 'เลย' ,'ก็' , 'ว่า' ,'หน้า' , 'ขอ' , 'กัน', 'บาท', 'แล้ว', 'กว่า', 'คุณ', 'อยู่', 'เห็น', 'ค่ะ',  '(' , ')' ]
screening_words=stopwords + removed_words
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    merged = merged + ',' + word
  return merged[1:]

In [66]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [67]:
df.tail()

,Review_ID,Review,Review_tokenized
15,16,ร้านเจ๊ไฝเป็นร้านที่เราอยากมาลองตั้งแต่ก่อนที่...,"ร้าน,เจ๊,ไฝ,เป็น,ร้าน,ที่,เรา,อยาก,มา,ลอง,ตั้ง..."
16,17,ไข่เจียวปูเจ๊ไฝ นี่มันปูห่อไข่ชัดๆๆ ข้างในคือม...,"ไข่เจียว,ปู,เจ๊,ไฝ, ,นี่,มันปู,ห่อ,ไข่,ชัด,ๆๆ,..."
17,18,คนที่บอกว่าเป็นร้านมิชลินที่ดังที่สุดในโลกไม่ใ...,"คน,ที่,บอ,กว่า,เป็น,ร้าน,มิชลิน,ที่,ดัง,ที่สุด..."
18,19,เป็นอีก 1 ร้าน ที่กระแสแรงมากๆในปีที่ผ่านมา ขอ...,"เป็น,อีก, ,1, ,ร้าน, ,ที่,กระแส,แรง,มาก,ๆ,ใน,ป..."
19,20,กินรอบสองก็ยังอร่อยเหมือนเดิม อร่อยไม่ไหวแล้ว ...,"กิน,รอบ,สอง,ก็,ยัง,อร่อย,เหมือนเดิม, ,อร่อย,ไม..."


In [68]:
documents=df['Review_tokenized'].to_list()
texts=[[text for text in doc.split(',')] for doc in documents]
dictionary=gensim.corpora.Dictionary(texts)

In [69]:
print(dictionary.token2id.keys())

dict_keys([' ', 'กรรเชียงปู', 'กับ', 'ก็', 'ของ', 'ขึ้น', 'ความเห็นส่วนตัว', 'คิด', 'คิว', 'คือ', 'จริง', 'จะ', 'จุดเด่น', 'ชาวต่างชาติ', 'ชื่อเรื่อง', 'ดัง', 'ดี', 'ตรง', 'ต้น', 'ทำให้', 'ที่', 'ปู', 'ผสม', 'มา', 'ยัดไส้', 'ยาว', 'รสชาติ', 'ร้าน', 'ลอง', 'ลูกค้า', 'ว่า', 'ว้าว', 'ส่วนมาก', 'อยาก', 'อันดับ', 'เจ๊', 'เนื้อ', 'เป็น', 'เมนู', 'เยอะ', 'เหมือน', 'แต่', 'ใน', 'ใส่', 'ใหญ่', 'ไข่', 'ไข่เจียว', 'ได้', 'ไทย', 'ไฝ', 'ไม่', 'ๆ', '-', '.', '/', '10', '200', '3', '38', '40', '400', '409', '44247', '535', '744', '93', '://', '=', '?', 'aa', 'aee', 'c', 'com', 'ct', 'd', 'e', 'f', 'https', 'ref', 'reviews', 'wongnai', 'www', 'กก', 'กิน', 'คนรู้จัก', 'จาน', 'ตั้งแต่', 'ต่อ', 'ทุก', 'นาน', 'บาท', 'ประถม', 'ประมาณ', 'ปี', 'ผ่าน', 'มัน', 'มี', 'ยัง', 'ย่อม', 'ราคา', 'รุ่น', 'รุ่นใหม่', 'วันนี้', 'สมัย', 'สาย', 'หรือ', 'อย่าง', 'อะไร', 'อาหาร', 'อ่ะ', 'อ่าน', 'เก๋า', 'เฉลี่ย', 'เปลี่ยนแปลง', 'เปิด', 'เป็นหนึ่ง', 'เพราะ', 'เรา', 'เรียน', 'แน่นอน', 'และ', 'ได้ที่', 'ได้มา', 'ไป', 'Vlog', 'ก

In [70]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies=[[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [71]:
num_topics = 3
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number fo passes through documents
iterations = 50
eval_every = 1 #Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary
temp = dictionary[0] # This is only to "load" the dicjjtionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize = chunksize, \
                                     alpha='auto', eta='auto', \
                                     iterations=iterations, num_topics=num_topics, \
                                     passes=passes, eval_every=eval_every)

CPU times: user 367 ms, sys: 2.52 ms, total: 369 ms
Wall time: 378 ms


In [72]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.041440  0.047952       1        1  69.271977
1     -0.036004 -0.050250       2        1  26.478361
0      0.077444  0.002298       3        1   4.249662, topic_info=     Term        Freq       Total Category  logprob  loglift
27   ร้าน   35.000000   35.000000  Default  30.0000  30.0000
0          199.000000  199.000000  Default  29.0000  29.0000
42     ใน   17.000000   17.000000  Default  28.0000  28.0000
23     มา   30.000000   30.000000  Default  27.0000  27.0000
9     คือ   12.000000   12.000000  Default  26.0000  26.0000
..    ...         ...         ...      ...      ...      ...
88    ทุก    0.368077    4.882245   Topic3  -5.4064   0.5733
506   ใช้    0.368069    2.612683   Topic3  -5.4064   1.1985
163  อยู่    0.368068    5.743768   Topic3  -5.4064   0.4107
15    ดัง    0.368041    3.159425   Topic3  -5.4065   1.0084
169  เห็น    0.368034    4.178274   Topic3  -5.4065   0.7289

[216 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.750886     
0         2  0.240284     
0         3  0.005006     
508       1  0.863457    (
408       1  0.922542    )
...     ...       ...  ...
49        2  0.213013   ไฝ
50        1  0.848899  ไม่
50        2  0.134037  ไม่
51        1  0.706004    ๆ
51        2  0.313780    ๆ

[198 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [73]:
model.show_topic(1)

[(' ', 0.09411228),
 ('ร้าน', 0.025251968),
 ('มา', 0.014230875),
 ('นี้', 0.01147862),
 ('ก็', 0.011478093),
 ('จะ', 0.011476818),
 ('ที่', 0.0114761805),
 ('ครับ', 0.010100146),
 ('ไป', 0.010098777),
 ('เป็น', 0.010098768)]

In [74]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [75]:
df.tail()

,Review_ID,Review,Review_tokenized,topics,score
15,16,ร้านเจ๊ไฝเป็นร้านที่เราอยากมาลองตั้งแต่ก่อนที่...,"ร้าน,เจ๊,ไฝ,เป็น,ร้าน,ที่,เรา,อยาก,มา,ลอง,ตั้ง...",2,0.999713
16,17,ไข่เจียวปูเจ๊ไฝ นี่มันปูห่อไข่ชัดๆๆ ข้างในคือม...,"ไข่เจียว,ปู,เจ๊,ไฝ, ,นี่,มันปู,ห่อ,ไข่,ชัด,ๆๆ,...",2,0.999665
17,18,คนที่บอกว่าเป็นร้านมิชลินที่ดังที่สุดในโลกไม่ใ...,"คน,ที่,บอ,กว่า,เป็น,ร้าน,มิชลิน,ที่,ดัง,ที่สุด...",2,0.999689
18,19,เป็นอีก 1 ร้าน ที่กระแสแรงมากๆในปีที่ผ่านมา ขอ...,"เป็น,อีก, ,1, ,ร้าน, ,ที่,กระแส,แรง,มาก,ๆ,ใน,ป...",2,0.999593
19,20,กินรอบสองก็ยังอร่อยเหมือนเดิม อร่อยไม่ไหวแล้ว ...,"กิน,รอบ,สอง,ก็,ยัง,อร่อย,เหมือนเดิม, ,อร่อย,ไม...",2,0.999701
